In [1]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-12-23 14:55:47--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-12-23 14:55:48--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-12-23 14:55:48--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [2]:
!java -version
!free -g

openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)
              total        used        free      shared  buff/cache   available
Mem:             12           0           6           0           5          11
Swap:             0           0           0


In [3]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/text-similarity/file1.csv
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/text-similarity/file2.csv


--2022-12-23 14:56:43--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/text-similarity/file1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 297 [text/plain]
Saving to: ‘file1.csv’

file1.csv           100%[===================>]     297  --.-KB/s    in 0s      

2022-12-23 14:56:43 (9.62 MB/s) - ‘file1.csv’ saved [297/297]

--2022-12-23 14:56:43--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/text-similarity/file2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP req

In [4]:
#!pip install --ignore-installed -q pyspark==2.4.5
#!gsutil cp gs://hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar /opt/conda/lib/python3.7/site-packages/pyspark/jars/
    
#!pip install --ignore-installed spark-nlp==2.5.1

In [5]:
!ps -ef | grep spark

root         587     241  0 14:56 ?        00:00:00 /bin/bash -c ps -ef | grep spark
root         589     587  0 14:56 ?        00:00:00 grep spark


In [6]:
import sparknlp
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import *
from pyspark.sql.functions import explode, col
from pyspark.sql.functions import from_unixtime, to_date, asc, year, udf, explode, split, col, desc, length, rank, dense_rank, avg, sum
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col, to_timestamp,date_format
from pyspark import StorageLevel
import pyspark.sql.functions as F
from sparknlp.pretrained import PretrainedPipeline
from collections import Counter
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from pyspark.ml.feature import Normalizer, SQLTransformer
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.sql.functions import monotonically_increasing_id

import pandas as pd


spark = sparknlp.start()

In [7]:
spark.version
!ps -ef | grep spark

root         604     241 91 14:56 ?        00:00:46 /usr/lib/jvm/java-11-openjdk-amd64/bin/java -cp /usr/local/lib/python3.8/dist-packages/pyspark/conf:/usr/local/lib/python3.8/dist-packages/pyspark/jars/* -Xmx16G org.apache.spark.deploy.SparkSubmit --conf spark.master=local[*] --conf spark.driver.memory=16G --conf spark.kryoserializer.buffer.max=2000M --conf spark.driver.maxResultSize=0 --conf spark.jars.packages=com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.6 --conf spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.app.name=Spark NLP pyspark-shell
root        1314     241  0 14:57 ?        00:00:00 /bin/bash -c ps -ef | grep spark
root        1316    1314  0 14:57 ?        00:00:00 grep spark


In [8]:
primaryCorpus = spark.read.option("header","true").csv("file1.csv")
secondaryCorpus = spark.read.option("header","true").csv("file2.csv")

In [10]:
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

sentence = SentenceDetector()\
      .setInputCols("document")\
      .setOutputCol("sentence")\
      .setExplodeSentences(False)

tokenizer = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

bertEmbeddings = BertEmbeddings\
 .pretrained('bert_base_cased', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
 .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["sentence", "bert"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

embeddingsFinisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings","bert"]) \
    .setOutputCols("sentence_embeddings_vectors", "bert_vectors") \
    .setOutputAsVector(True)\
    .setCleanAnnotations(False)


explodeVectors = SQLTransformer() \
.setStatement("SELECT EXPLODE(sentence_embeddings_vectors) AS features, * FROM __THIS__")

vectorNormalizer = Normalizer() \
    .setInputCol("features") \
    .setOutputCol("normFeatures") \
    .setP(1.0)

similartyChecker = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=6.0,numHashTables=6)
    

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [11]:
pipeline = Pipeline() \
      .setStages([documentAssembler,
        sentence,
        tokenizer,
        bertEmbeddings,
        embeddingsSentence,
        embeddingsFinisher,
        explodeVectors,
        vectorNormalizer,
        similartyChecker])

In [12]:
pipelineModel = pipeline.fit(primaryCorpus)
primaryDF = pipelineModel.transform(primaryCorpus)
secondaryDF = pipelineModel.transform(secondaryCorpus)

In [13]:
dfA = primaryDF.select("text","features","normFeatures").withColumn("lookupKey", md5("text")).withColumn("id",monotonically_increasing_id())
dfA.show()

+--------------------+--------------------+--------------------+--------------------+---+
|                text|            features|        normFeatures|           lookupKey| id|
+--------------------+--------------------+--------------------+--------------------+---+
|Wall Decals Lamp ...|[0.04242564737796...|[2.48993627607806...|bbc5a89d7cf3354ea...|  0|
|iphone charger ph...|[0.37093448638916...|[0.00200630526885...|37c2b6ab956f9ebd6...|  1|
+--------------------+--------------------+--------------------+--------------------+---+



In [14]:
dfB = secondaryDF.select("text","features","normFeatures").withColumn("id",monotonically_increasing_id())
dfB.show()

+--------------------+--------------------+--------------------+---+
|                text|            features|        normFeatures| id|
+--------------------+--------------------+--------------------+---+
|Curtains & Valanc...|[0.30033871531486...|[0.00192763000744...|  0|
|iphone case Apple...|[0.44015255570411...|[0.00236218518925...|  1|
+--------------------+--------------------+--------------------+---+



In [15]:
#print("Approximately joining dfA and dfB :")
pipelineModel.stages[8].approxSimilarityJoin(dfA, dfB, 100, distCol="distance")\
     .where(col("datasetA.id") == col("datasetB.id")) \
     .select(col("datasetA.text").alias("idA"), \
            col("datasetB.text").alias("idB"), \
            col("distance")).show()


/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+--------------------+------------------+
|                 idA|                 idB|          distance|
+--------------------+--------------------+------------------+
|iphone charger ph...|iphone case Apple...| 5.666233511624179|
|Wall Decals Lamp ...|Curtains & Valanc...|3.7816639073044893|
+--------------------+--------------------+------------------+



## Approach 2

In [16]:
from pyspark.sql.functions import PandasUDFType, pandas_udf
import pyspark.sql.functions as F

dfA = dfA.withColumnRenamed('text','primaryText').withColumnRenamed('features', 'primaryFeatures')

dfB = dfB.withColumnRenamed('text','secondaryText').withColumnRenamed('features', 'secondaryFeatures')

joinedDF = dfA.join(dfB, "id", "inner").drop("id","normFeatures")

joinedDF.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|         primaryText|     primaryFeatures|           lookupKey|       secondaryText|   secondaryFeatures|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Wall Decals Lamp ...|[0.04242564737796...|bbc5a89d7cf3354ea...|Curtains & Valanc...|[0.30033871531486...|
|iphone charger ph...|[0.37093448638916...|37c2b6ab956f9ebd6...|iphone case Apple...|[0.44015255570411...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [17]:
from scipy.spatial.distance import cosine

finalDF = joinedDF.toPandas()

finalDF['cosine'] = finalDF.apply(lambda row: 1-cosine(row['primaryFeatures'], row['secondaryFeatures']), axis=1)
finalDF

,primaryText,primaryFeatures,lookupKey,secondaryText,secondaryFeatures,cosine
0,Wall Decals Lamp Shades Armchairs Bed Sheets N...,"[0.042425647377967834, -0.226881206035614, -0....",bbc5a89d7cf3354ea4887c3690404ad8,Curtains & Valances Wall Decals & Stickers Bed...,"[0.3003387153148651, -0.022465573623776436, -0...",0.942328
1,iphone charger phone Gift case iPhone holder s...,"[0.37093448638916016, 0.07500777393579483, -0....",37c2b6ab956f9ebd6dccebd7623bf8c1,iphone case Apple ipod,"[0.4401525557041168, -0.09592525660991669, 0.0...",0.885493
